In [5]:
!pip install trafilatura

  Using cached babel-2.17.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.6 MB/s eta 0:00:00a 0:00:01
Using cached babel-2.17.0-py3-none-any.whl (10.2 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
  Attempting uninstall: lxml
    Found existing installation: lxml 5.2.1
    Uninstalling lxml-5.2.1:
      Successfully uninstalled lxml-5.2.1
  Attempting uninstall: charset_normalizer
    Found existing installation: charset-normalizer 3.3.2
    Uninstalling charset-normalizer-3.3.2:
      Successfully uninstalled charset-normalizer-3.3.2
  Attempting uninstall: babel
    Found existing installation: Babel 2.11.0
    Uninstalling Babel-

In [49]:
import pandas as pd
from trafilatura import fetch_url, extract
import json

In [51]:
df1 = pd.read_csv('/Users/caio.mello/Documents/NetzDG_project/NetzDG_data/tichyseinblick_scraping/tichys_Gesetz+zur+Verbesserung+der+Rechtsdurchsetzung+in+sozialen+Netzwerken.csv')
df2 = pd.read_csv('/Users/caio.mello/Documents/NetzDG_project/NetzDG_data/tichyseinblick_scraping/tichys_Facebook-Gesetz.csv')
df3 = pd.read_csv('/Users/caio.mello/Documents/NetzDG_project/NetzDG_data/tichyseinblick_scraping/tichys_NetzDG.csv')
df4 = pd.read_csv('/Users/caio.mello/Documents/NetzDG_project/NetzDG_data/tichyseinblick_scraping/tichys_Netzwerkdurchsetzungsgesetz.csv')

df_urls = pd.concat([df1, df2, df3, df4], axis=0)



In [54]:
df3.head()

,Unnamed: 0,URL
0,0,https://www.tichyseinblick.de/kolumnen/aus-all...
1,1,https://www.tichyseinblick.de/meinungen/meldes...
2,2,https://www.tichyseinblick.de/meinungen/schlaf...
3,3,https://www.tichyseinblick.de/meinungen/netzdg...
4,4,https://www.tichyseinblick.de/kolumnen/aus-all...


In [58]:
df_urls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386 entries, 0 to 202
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  386 non-null    int64 
 1   URL         386 non-null    object
dtypes: int64(1), object(1)
memory usage: 9.0+ KB


In [66]:
value_counts = df_urls['URL'].value_counts()
value_counts


URL
https://www.tichyseinblick.de/video/interview/verfassungssrichter-staatsstreich-hinterzimmer/                                                  4
https://www.tichyseinblick.de/meinungen/maas-netzwerkdurchsetzungsgesetz-netzdg/                                                               4
https://www.tichyseinblick.de/daili-es-sentials/der-wissenschaftliche-dienst-des-bundestages-netzwerkdurchsetzungsgesetz-verfassungswidrig/    3
https://www.tichyseinblick.de/daili-es-sentials/verfassungsschutz-afd-programm-verdachstfall/                                                  3
https://www.tichyseinblick.de/daili-es-sentials/verlegerverband-vdz-gegen-netzwerkdurchsetzungsgesetz/                                         3
                                                                                                                                              ..
https://www.tichyseinblick.de/meinungen/schlaf-buerger-schlaf/                                                                

In [59]:
extracted_data_list = []
processed_urls = []
processed_urls_set = set(processed_urls)

In [62]:
progress_filename = 'extracted_data_progress.json'

for index, row in df_urls.iterrows():
    url = row['URL']

    if url in processed_urls_set:
        print(f"URL already processed, skipping: {url}")
        continue # Skip to the next URL

    print(f"Processing URL: {url}")
    try:
        downloaded_html = fetch_url(url)
        if downloaded_html:
            extracted_data = extract(downloaded_html, output_format="json", with_metadata=True)
            if extracted_data:
                extracted_data_list.append(extracted_data)
                # Save progress after each successful extraction
                try:
                    with open(progress_filename, 'w') as f:
                        json.dump(extracted_data_list, f, indent=4)
                    # Add the successfully processed URL to the set
                    processed_urls_set.add(url)
                    print(f"Progress saved to {progress_filename}")
                except IOError as e:
                    print(f"Error saving progress to {progress_filename}: {e}")
            else:
                print(f"Could not extract data from: {url}")
        else:
            print(f"Could not download content from: {url}")
    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")


URL already processed, skipping: https://www.tichyseinblick.de/kolumnen/josef-kraus-lernen-und-bildung/maas-macht-mobil/
URL already processed, skipping: https://www.tichyseinblick.de/video/interview/verfassungssrichter-staatsstreich-hinterzimmer/
URL already processed, skipping: https://www.tichyseinblick.de/kolumnen/josef-kraus-lernen-und-bildung/aussenminister-heiko-maas-gendert-auch-buecherverbrenner/
URL already processed, skipping: https://www.tichyseinblick.de/daili-es-sentials/netzdg-afd-und-die-linke/
URL already processed, skipping: https://www.tichyseinblick.de/daili-es-sentials/auch-ansteckend-hass-und-hetze/
URL already processed, skipping: https://www.tichyseinblick.de/daili-es-sentials/netzdg-pressemitteilungen-fdp-nicola-beer-und-die-piraten/
URL already processed, skipping: https://www.tichyseinblick.de/feuilleton/buecher/douglas-murray-vorwort-an-seine-deutschen-leserdouglas-murray-vorwort-an-seine-deutschen-leser/
URL already processed, skipping: https://www.tichysei

In [ ]:
# Convert the list of dictionaries into a pandas DataFrame
df_extracted = pd.DataFrame(extracted_data_list)

# Save the DataFrame to a CSV file without the index
output_filename = 'extracted_tichys_data.csv'
df_extracted.to_csv(output_filename, index=False)

print(f"Extracted data saved to {output_filename}")

# Print the head and info of the created DataFrame
print("\nHead of the extracted data DataFrame:")
display(df_extracted.head())

print("\nInfo of the extracted data DataFrame:")
df_extracted.info()

Extracted data saved to extracted_philosophia_perennis_data.csv

Head of the extracted data DataFrame:


,0
0,"{""title"": ""Maas macht mobil"", ""author"": ""Sofia..."
1,"{""title"": ""Der verhinderte Staatsstreich: Sieg..."
2,"{""title"": ""Außenminister Heiko Maas gendert au..."
3,"{""title"": ""NetzDG - AfD und Die Linke"", ""autho..."
4,"{""title"": ""Auch ansteckend: „Hass und Hetze“"",..."



Info of the extracted data DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       284 non-null    object
dtypes: object(1)
memory usage: 2.3+ KB


In [46]:
df_extracted

,0
0,"{""title"": ""Zensur: Jetzt geht es \""Telegram\"" ..."
1,"{""title"": ""Interview mit Rechtsanwalt Dr. Stah..."
2,"{""title"": ""Bundestag: Bekämpfung von News, die..."
3,"{""title"": ""\""Sie haben schlicht Angst vor eine..."
4,"{""title"": ""Deutschland erweckt fatale Traditio..."
...,...
178,"{""title"": ""Jagd auf Facebook: Wie die Grünen d..."
179,"{""title"": ""Gescheiterte Anbiederungsversuche m..."
180,"{""title"": ""Imad Karim und PP erneut Opfer von ..."
181,"{""title"": ""Limburg: Täter der LKW-Terrorfahrt ..."


In [64]:
df_extracted.to_csv('/Users/caio.mello/Documents/NetzDG_project/NetzDG_data/tichyseinblick_scraping/extracted_tichyseinblick_metadata&text.csv', index=False)

In [65]:
df_extracted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       284 non-null    object
dtypes: object(1)
memory usage: 2.3+ KB
